In [20]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 方向从 0° 到 360°
angles = np.linspace(0, 2 * np.pi, 360)
# 主方向
grid_orientation = 35.0 

# 参考论文中的六角对称性，模拟每六十度为一个周期
activations = [np.cos(6 * (angle - grid_orientation)) for angle in angles]

angles_degrees = np.degrees(angles)

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=angles_degrees,
    y=activations,
    mode='lines+markers',
    name='Grid Cell Activation',
    line=dict(width=2),
    marker=dict(size=6)
))
fig.add_shape(
    type='line',
    x0=0,
    x1=360,
    y0=0,
    y1=0,
    line=dict(color='gray', dash='dash'),
    name='Zero Line'
)

fig.update_layout(
    title="Grid Cell Activation vs. Direction (60° Periodicity)",
    xaxis_title="Direction (°)",
    yaxis_title="Activation Level",
    xaxis=dict(
        tickmode='array',
        tickvals=list(range(0, 361, 60)),  
        range=[0, 360]
    ),
    yaxis=dict(
        zeroline=True,
        zerolinecolor='gray'
    ),
    template="plotly_white",
    showlegend=True
)
fig.show()

In [21]:
grid_size = 100 
phi = np.linspace(0, 2 * np.pi, 6, endpoint=False) 
angles = np.linspace(0, 2 * np.pi, 360)  
grid_orientation = 35  # 主方向角度

x, y = np.meshgrid(np.linspace(-10, 10, grid_size), np.linspace(-10, 10, grid_size))
grid_cell_activation = np.cos(6 * (np.arctan2(y, x) - np.radians(grid_orientation)))

# 模拟穿越路径
traverse_activations = []
for angle in angles:
    path_x = np.linspace(-10, 10, grid_size) * np.cos(angle)
    path_y = np.linspace(-10, 10, grid_size) * np.sin(angle)
    path_activation = []
    for px, py in zip(path_x, path_y):
        # 找到路径上的最近激活值
        xi = np.abs(x[0] - px).argmin()  
        yi = np.abs(y[:, 0] - py).argmin()  
        path_activation.append(grid_cell_activation[yi, xi])
    traverse_activations.append(np.sum(path_activation))

heatmap = go.Heatmap(
    z=grid_cell_activation,
    x=np.linspace(-10, 10, grid_size),
    y=np.linspace(-10, 10, grid_size),
    colorscale="Viridis",
    colorbar=dict(title="Activation Level"),
)

# 激活值变化
line_plot = go.Scatter(
    x=np.degrees(angles),  
    y=traverse_activations,
    mode="lines+markers",
    name="Activation Sum",
    line=dict(width=2),
    marker=dict(size=6),
)

layout_heatmap = go.Layout(
    title="Grid Cell Activation Map",
    xaxis=dict(title="X", showgrid=False),
    yaxis=dict(title="Y", showgrid=False),
    template="plotly_white",
)
layout_line_plot = go.Layout(
    title="Activation Along Traversal Direction",
    xaxis=dict(title="Direction (°)", tickmode="array", tickvals=list(range(0, 361, 60))),
    yaxis=dict(title="Activation Sum", zeroline=True, zerolinecolor="gray"),
    template="plotly_white",
)

fig = make_subplots(
    rows=1,
    cols=2,
    subplot_titles=("Grid Cell Activation Map", "Activation Along Traversal Direction"),
    column_widths=[0.5, 0.5],
)

fig.add_trace(heatmap, row=1, col=1)
fig.add_trace(line_plot, row=1, col=2)

fig.update_layout(height=600, width=1000, title_text="Grid Cell Activation and Traversal Patterns")
fig.show()